### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [3]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [5]:
from skopt import BayesSearchCV

# Define the function to create the model, required for KerasClassifier
def create_model(neurons=32, layers=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation='relu'))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Adjust output for binary classification
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Wrap the Keras model with KerasClassifier
model = KerasClassifier(layers=1,neurons=32,build_fn=create_model, verbose=0)

# Define the parameter search space
search_space = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

# Create the BayesSearchCV object
bayes_search = BayesSearchCV(estimator=model, search_spaces=search_space, n_iter=20, cv=3, n_jobs=-1, verbose=1)

# Fit the model with Bayesian optimization
bayes_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best: %f using %s" % (bayes_search.best_score_, bayes_search.best_params_))

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/skopt/space/space.py:116: UserWarning: Dimension [1, 2] was inferred to Integer(low=1, high=2, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(1, 2), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/skopt/space/space.py:116: UserWarning: Dimension [50, 100] was inferred to Integer(low=50, high=100, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(50, 100), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/skopt/space/space.py:116: UserWarning: Dimension [50, 100] was inferred to Integer(low=50, high=100, prior='uniform', transform

/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer usi

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will 

...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [50, 1, 64] before, using random point [89, 1, 128]
  warnings.warn(
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future rele

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pa

Best: 0.857875 using OrderedDict([('epochs', 54), ('layers', 1), ('neurons', 16)])


In [30]:
### Retrain model using best params ()

# Retrieve the best parameters
best_params = bayes_search.best_params_
best_score = bayes_search.best_score_

# Define your model-building function if it requires any custom parameters
def create_model(epochs=32, layers=1):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))  # Input layer
    for _ in range(best_params['layers']):
        model.add(Dense(32, activation='relu'))  # Add hidden layers based on best_params['layers']
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Rebuild the model with the best parameters
model = create_model(epochs=best_params['epochs'], layers=best_params['layers'])

# Fit the model using the best number of epochs and any other necessary parameters
model.fit(X_train, y_train, epochs=best_params['epochs'], verbose=1)

# Save the model
model.save('best_model.h5')

# Save best model metadata in a dictionary
model_data = {
    'best_params': best_params,
    'best_score': best_score,
    'cv_results': bayes_search.cv_results_
}

# Use pickle to save model metadata
with open("best_model_metadata.pkl", "wb") as file:
    pickle.dump(model_data, file)

Epoch 1/54


/opt/miniconda3/envs/churn_modeling/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7484 - loss: 0.5130
Epoch 2/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8337 - loss: 0.3987
Epoch 3/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8555 - loss: 0.3504
Epoch 4/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8464 - loss: 0.3580
Epoch 5/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8586 - loss: 0.3392
Epoch 6/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8642 - loss: 0.3325
Epoch 7/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8679 - loss: 0.3300
Epoch 8/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8629 - loss: 0.3262
Epoch 9/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8629 - loss: 0.3325
Epoch 10/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8741 - loss: 0.3060
Epoch 11/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8730 - loss: 0.3122
Epoch 12/54
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy

In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Make predictions on the test set
y_pred = (best_model.predict(X_test) > 0.5).astype("int32")  # Binarize predictions for binary classification

# Calculate additional metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test F1 Score: {f1}")

Test Loss: 0.36901339888572693
Test Accuracy: 0.8539999723434448
Test Precision: 0.7341269841269841
Test Recall: 0.4707379134860051
Test F1 Score: 0.5736434108527132


In [40]:
# load best model and metadata (as needed)
with open("best_model_metadata.pkl", "rb") as file:
    model_data = pickle.load(file)

# load model
model = tf.keras.models.load_model('best_model.h5')

# Access the saved model metadata
best_params = model_data['best_params']
best_score = model_data['best_score']

print("Loaded model with parameters:", best_params)
print("Loaded model with best score:", best_score)

Loaded model with parameters: OrderedDict([('epochs', 54), ('layers', 1), ('neurons', 16)])
Loaded model with best score: 0.8578745267494227


In [39]:
# # Perform grid search

## Define a function to create the model and try different parameters(KerasClassifier)
# def create_model(neurons=32,layers=1):
#     model=Sequential()
#     model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

#     for _ in range(layers-1):
#         model.add(Dense(neurons,activation='relu'))

#     model.add(Dense(1,activation='sigmoid'))
#     model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

#     return model


# ## Create a Keras classifier
# model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

# # Define the grid search parameters
# param_grid = {
#     'neurons': [16, 32, 64, 128],
#     'layers': [1, 2],
#     'epochs': [50, 100]
# }

# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
# grid_result = grid.fit(X_train, y_train)

# # Print the best parameters
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))